# Support Vector Machine
MSc in Statistical Science\
University of Oxford\
Group-assessed practical\
HT 2024


In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import accuracy_score, zero_one_loss, make_scorer, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))
# The scikit-learn version is 0.15.2.

The scikit-learn version is 1.2.2.


## Load data

In [2]:
# Load the training data and the test inputs
X_train = pd.read_csv('X_train.csv', index_col = 0, header=[0, 1, 2]) # inputs of the training set
y_train = pd.read_csv('y_train.csv', index_col=0).squeeze('columns').to_numpy() # outputs of the training set
X_test = pd.read_csv('X_test.csv', index_col = 0, header=[0, 1, 2]) # inputs of the test set

In [3]:
X = X_train
y = y_train

# Split the data into training and test sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# Scale the features using StandardScaler
scalerx = StandardScaler()
X_train = scalerx.fit_transform(X_train)
X_val = scalerx.transform(X_val)

## Reduce dimension

In [4]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_sc = scaler.transform(X_train)
X_val_sc = scaler.transform(X_val)

In [5]:
lda = LinearDiscriminantAnalysis(n_components=None)
lda.fit(X_train_sc, y_train)
X_train_LDA = lda.transform(X_train_sc)
X_val_LDA = lda.transform(X_val_sc)

In [6]:
print(X_train_LDA.shape)
print(X_val_LDA.shape)

(4800, 7)
(1200, 7)


## SVM

In [13]:
param_grid = [{'C': [0.01, 0.1, 1, 10, 100], 'kernel': ['poly'], 'degree': [0, 1, 2, 3, 4, 5], 'gamma': ['scale', 'auto']},
              {'C': [0.01, 0.1, 1, 10, 100], 'kernel': ['rbf', 'sigmoid'], 'gamma': ['scale', 'auto']}]

In [14]:
svc = SVC()
scorer = make_scorer(zero_one_loss, greater_is_better=False)
grid_search = GridSearchCV(svc, param_grid=param_grid, cv=5, scoring=scorer, n_jobs=-1, verbose=10)
grid_search.fit(X_train_LDA, y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid=[{'C': [0.01, 0.1, 1, 10, 100],
                          'degree': [0, 1, 2, 3, 4, 5],
                          'gamma': ['scale', 'auto'], 'kernel': ['poly']},
                         {'C': [0.01, 0.1, 1, 10, 100],
                          'gamma': ['scale', 'auto'],
                          'kernel': ['rbf', 'sigmoid']}],
             scoring=make_scorer(zero_one_loss, greater_is_better=False),
             verbose=10)

In [15]:
print(grid_search.best_params_)
print(grid_search.best_score_)

{'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}
-0.2975


In [16]:
svc_best = SVC(C=0.1, kernel='rbf', gamma='scale')
svc_best.fit(X_train_LDA, y_train)
val_score = svc_best.score(X_val_LDA, y_val)
print(val_score)

0.5691666666666667
